In [1]:
#pip install llama-index-llms-huggingface

In [2]:
#pip install llama-index

In [3]:
# pip install llama-index-embeddings-huggingface

In [4]:
# pip install llama-index-embedding-instructor

In [5]:
from llama_index.core import SimpleDirectoryReader

directory_reader = SimpleDirectoryReader(input_dir='D:\RAG\RAG App using LLamaindex and Mistral')
directory_reader.load_data()

[Document(id_='da223c58-629b-4c1d-9684-fab71aa41bf1', embedding=None, metadata={'page_label': '1', 'file_name': 'MachineTranslationwithAttention.pdf', 'file_path': 'D:\\RAG\\RAG App using LLamaindex and Mistral\\MachineTranslationwithAttention.pdf', 'file_type': 'application/pdf', 'file_size': 482143, 'creation_date': '2025-07-22', 'last_modified_date': '2025-07-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/355917108\nNeural Machine Translation with Attention\nTechnical Report · August 2021\nDOI: 10.13140/RG.2.2.29381.37607

In [6]:
documents = directory_reader.load_data()

In [7]:
from llama_index.core import PromptTemplate

system_prompt = """ <|SYSTEM|># You are a QA Assistant. Your goal is to answer question accurately
as possible based on the context and your knowledge.
"""
query_wrapper_prompts = PromptTemplate("<|USER|>{query}<|ASSISTANT|>")

In [8]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [9]:
from huggingface_hub import login

login(os.getenv("HF_API_KEY"))


In [10]:
# mistralai/Mistral-7B-v0.1

In [11]:
# pip install hf_xet

In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    query_wrapper_prompt=query_wrapper_prompts,
    system_prompt=system_prompt,
    tokenizer_name="mistralai/Mistral-7B-v0.1",
    model_name="mistralai/Mistral-7B-v0.1",
    device_map='auto',
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={'max_length':4096,'pad_token_id': 2},
    model_kwargs={'torch_dtype':torch.float32},
    generate_kwargs={'temperature':0.5,'do_sample':False}

)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [14]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(
    model_name = 'sentence-transformers/all-mpnet-base-v2',
)

In [15]:
from llama_index.core.node_parser import SentenceSplitter

node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)

In [18]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = node_parser
Settings.num_output = 512
Settings.context_window = 3900

In [19]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

In [20]:
'What is the main topic of the document?'

'What is the main topic of the document?'

In [21]:
query_engine = index.as_query_engine(streaming=True)

In [22]:
response = query_engine.query("What is attention?")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(response.source_nodes[0].text)

the last hidden unit of the
encoder RNN is used as a context vector for the
decoder, which becomes difﬁcult for the decoder
to summarise large input sequence at once. This
also poses a problem where the encoder is not able
to memorize the words coming at the beginning
of the sentences, which leads to poor translation
of the source sentence. The Attention mechanism
just addresses this issue, by retaining and utilising
all the hidden state of the input sentence during the
decoding phase.
During the decoding phase, the model creates
an alignment between each time step of the de-
coder output and all of the encoder hidden state.
We need to learn this alignment. Each output of
the decoder can selectively pick out speciﬁc ele-
ments from the sequence to produce the output.
So, this allows the model to focus and pay more
”Attention” to the relevant part of the input se-
quence.
The ﬁrst attention model was proposed by Bah-
danau et al. (2014), there are several other types of
attention propos

: 

In [ ]:
# response2 = query_engine.query("What is the main topic of the document?")

In [ ]:
# query_engine.query("What is attention?")